In [1]:
import cv2
import numpy as np
import glob
from scipy.ndimage import convolve
from skimage.filters import frangi, sato, threshold_otsu
from skimage.morphology import remove_small_objects, binary_closing, disk, skeletonize, binary_dilation, reconstruction
from skimage.measure import label, regionprops
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:

def turn_grey(image):
    """
    Turns the input image to greyscale
    
    :param image: image to be turned gray
    """
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def to_uint8(binary):
    return binary.astype(np.uint8) * 255

def resize_image(image, scale_percent):
    """
    Scales input image using a percentage
    
    :param image: Input image
    :param scale_percent: ratio to which the image is being resized
    """
    width = image.shape[1]
    height = image.shape[0]
    new_width = int(width * scale_percent / 100)
    new_height = int(height * scale_percent / 100)
    rezised_image = cv2.resize(image, (new_width, new_height))
    return rezised_image

def resize_image_to_side(image, side_length):
    """
    Resizes input image to desired side length 
    
    :param image: Input image to be resized
    :param side_length: Desired side length
    """
    height, width = image.shape[:2]
    if width >= height:
        scale_percent = (side_length / width) * 100
    else:
        scale_percent = (side_length / height) * 100
    return resize_image(image, scale_percent)

def mild_smooth(img, method="gaussian", ksize=3, sigma=1.0):
    
    if method == "gaussian":
        return cv2.GaussianBlur(img, (ksize, ksize), sigma)
    
    elif method == "median":
        return cv2.medianBlur(img, ksize)
    
    elif method == "bilateral":
        # Diameter = ksize, sigmaColor = sigma, sigmaSpace = sigma
        return cv2.bilateralFilter(img, d=ksize, sigmaColor=sigma*25, sigmaSpace=sigma*25)
    
    else:
        raise ValueError("Method must be 'gaussian', 'median', or 'bilateral'.")
    
def apply_clahe(img):
    """
    Applies clahe with pre-tested fairly well working parameters to enhance contrast
    
    :param img: Image to be boosted in contrast
    """
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    return clahe.apply(img)

def apply_frangi(img):
    """
    Tries to recognize vessel like structures using frangi
    
    :param img: Image to be vesselized 
    """
    # skimage expects float in [0,1]
    img_norm = img.astype(np.float32) / 255.0
    return frangi(img_norm,
                    scale_range=(1, 2),
                #   scale_step=2,
                #   beta=0.5,
                #   alpha=0.5,
                #   gamma=15,
                    black_ridges=True
                  )

def _order_points_clockwise(pts):
    """
    Order 4 points consistently: top-left, top-right, bottom-right, bottom-left.
    Input: pts as numpy array shape (4,2) (float or int).
    """
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]      # top-left has smallest sum (x+y)
    rect[2] = pts[np.argmax(s)]      # bottom-right has largest sum
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]   # top-right has smallest (x-y)
    rect[3] = pts[np.argmax(diff)]   # bottom-left has largest (x-y)
    return rect

def normalize(img):
    return cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

def extract_square_glass_bmp(image_or_path,
                             output_path=None,
                             margin=0,
                             min_area_ratio=0.001,
                             debug=False):
    """
    Detect the largest square/rectangular sample in the input (preserving original resolution),
    deskew it and return (and optionally save) a square BMP image of the sample.

    Parameters:
    - image_or_path: numpy array (grayscale or BGR) OR path string to image file.
    - output_path: if provided, save the result as BMP at this path; if None, do not save.
    - margin: additional pixels to add around the detected square in the output (keeps resolution).
    - min_area_ratio: ignore tiny contours smaller than this fraction of image area.
    - debug: if True, returns a tuple (warped, debug_vis) where debug_vis draws the detected box.

    Returns:
    - warped (numpy uint8 image) or (warped, debug_vis) if debug=True.
    Notes:
    - The output size equals the detected side length (in original pixels) + 2*margin -> no resampling down/up.
    - If detection fails, the original image is returned (and saved if output_path provided).
    """

    # Load if path string given
    if isinstance(image_or_path, (str,)):
        img = cv2.imread(image_or_path, cv2.IMREAD_UNCHANGED)
        if img is None:
            raise FileNotFoundError(f"Could not load image: {image_or_path}")
    else:
        img = image_or_path.copy()

    # Keep original for return/save
    orig = img.copy()

    # Ensure grayscale for edge detection
    if orig.ndim == 3:
        gray = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
    else:
        gray = orig

    h, w = gray.shape[:2]
    image_area = float(w * h)

    # Preprocess: slight blur to reduce noise, then edge detection
    blurred = cv2.GaussianBlur(gray, (5,5), 0)

    # Use automatic Canny thresholds based on median (robust to exposure)
    med = np.median(blurred)
    lower = int(max(0, 0.66 * med))
    upper = int(min(255, 1.33 * med))
    edges = cv2.Canny(blurred, lower, upper)

    # Close small gaps on the border lines (helps faint borders)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find external contours
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        # nothing found -> return original (optionally save)
        if output_path:
            cv2.imwrite(output_path, orig)
        if debug:
            debug_vis = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
            return orig, debug_vis
        return orig

    # Filter and pick the best contour: largest by area above threshold
    candidates = []
    for c in contours:
        area = cv2.contourArea(c)
        if area < image_area * min_area_ratio:
            continue
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        candidates.append((area, approx, c))

    if not candidates:
        # fallback to largest contour without area filter
        largest = max(contours, key=cv2.contourArea)
        rect = cv2.minAreaRect(largest)
        box = cv2.boxPoints(rect)
    else:
        # prefer a 4-vertex polygon with largest area
        candidates.sort(key=lambda x: (x[0], -len(x[1])), reverse=True)
        best_area, best_approx, best_contour = candidates[0]
        if best_approx.shape[0] == 4:
            box = best_approx.reshape(4,2).astype("float32")
        else:
            rect = cv2.minAreaRect(best_contour)
            box = cv2.boxPoints(rect)

    box = np.asarray(box, dtype="float32")
    rect = _order_points_clockwise(box)

    # Compute side length (use the maximum of widths/heights to obtain a square)
    (tl, tr, br, bl) = rect
    widthA = np.linalg.norm(br - bl)
    widthB = np.linalg.norm(tr - tl)
    heightA = np.linalg.norm(tr - br)
    heightB = np.linalg.norm(tl - bl)
    maxSide = int(max(widthA, widthB, heightA, heightB))

    if maxSide <= 0:
        # Something went wrong; return original
        if output_path:
            cv2.imwrite(output_path, orig)
        if debug:
            debug_vis = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
            return orig, debug_vis
        return orig

    # - - - replace the perspective warp below with the rotation-based crop - - -
    # Use the detected box to compute a robust rotated rect (center, size, angle)
    rect_rot = cv2.minAreaRect(box)  # ((cx,cy),(w,h),angle)
    (cx, cy), (rw, rh), angle = rect_rot

    # OpenCV returns angle such that width may be the shorter side; ensure we rotate so longer side is horizontal
    if rw < rh:
        angle += 90.0
        rw, rh = rh, rw

    # Build rotation matrix around full-image center of rectangle and rotate full-res image (no scaling)
    center = (float(cx), float(cy))
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(orig, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)

    # Transform detected box points into rotated image coordinates and crop tight bounding box
    box_pts = cv2.transform(np.array([box]), M)[0]
    x_min = max(0, int(np.floor(box_pts[:, 0].min())) - margin)
    x_max = min(rotated.shape[1], int(np.ceil(box_pts[:, 0].max())) + margin)
    y_min = max(0, int(np.floor(box_pts[:, 1].min())) - margin)
    y_max = min(rotated.shape[0], int(np.ceil(box_pts[:, 1].max())) + margin)

    cropped = rotated[y_min:y_max, x_min:x_max]

    # Pad to exact square without resampling (centered)
    ch, cw = cropped.shape[:2]
    side = max(ch, cw)
    top = (side - ch) // 2
    bottom = side - ch - top
    left = (side - cw) // 2
    right = side - cw - left
    final = cv2.copyMakeBorder(cropped, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)

    # Ensure uint8 (BMP requires 8-bit channels)
    if final.dtype != np.uint8:
        final = normalize(final).astype(np.uint8)

    # Save if requested (BMP format)
    if output_path:
        out_path = str(output_path)
        if not out_path.lower().endswith(".bmp"):
            out_path = out_path + ".bmp"
        cv2.imwrite(out_path, final)

    if debug:
        debug_vis = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
        int_box = box.astype(int)
        cv2.polylines(debug_vis, [int_box.reshape(-1,1,2)], True, (0,255,0), 3)
        return final, debug_vis

    return cv2.rotate(final, cv2.ROTATE_90_CLOCKWISE)  # rotate 90 degree clockwise to get original orientation

def clean_edges(binary, margin = 40):
    """
    Cleans all elements from the edges of a binary mask following a margin
    
    :param binary: binary mask to be cleaned
    :param margin: how much should it be removed
    """
    h, w = binary.shape  

    # Create a mask that’s True everywhere except near the border
    mask = np.zeros_like(binary, dtype=bool)
    mask[margin:h - margin, margin:w - margin] = True

    # Keep only central detections
    return binary & mask

def generate_mask(img):
    """
    Pipeline to generate a binary mask of ruptures from image using sato instead of frangi.
    The last step cleans too many pixels away and thus is not very good
    
    :param img: Image to be pipelined
    """
    # Contrast Enhancement
    clahe_img = apply_clahe(img)
    # to float32
    img = clahe_img.astype(np.float32) / 255.0
    # Sato threshholding
    v_sato = sato(img, sigmas=range(1, 8), black_ridges=True)
    # Otsu autothesholding
    thresh = threshold_otsu(v_sato)
    binary = v_sato > thresh
    # Morphological cleaning
    binary = remove_small_objects(binary, 100)
    binary = binary_closing(binary, disk(2))

    labeled = label(binary)
    filtered = np.zeros_like(binary)

    for region in regionprops(labeled):
        # region.eccentricity ∈ [0,1], close to 1 = elongated (cracks)
        # region.solidity close to 1 = solid blob (bubbles)
        if region.eccentricity > 0.9 and region.solidity < 0.95:
            filtered[labeled == region.label] = 1

    return filtered.astype(bool)


# Generate mask without the last step of filtering by region properties
def generate_mask_incomplete(img):
    """
    Another pipeline to generate a mask, without one step of cleaning which was taking off too many elements
    
    :param img: Description
    """

    # Contrast Enhancement
    clahe_img = apply_clahe(img)
    # to float32
    img = clahe_img.astype(np.float32) / 255.0
    # Sato threshholding
    v_sato = sato(img, sigmas=range(1, 8), black_ridges=True)
    # Otsu autothesholding
    thresh = threshold_otsu(v_sato)
    binary = v_sato > thresh
    # Morphological cleaning
    binary = remove_small_objects(binary, 100)
    binary = binary_closing(binary, disk(2))

    return binary.astype(bool)



In [15]:
#Refined pipelines
def generate_mask_trans(img):
    """
    Transmission channel pipeline.
    Very conservative, used mostly as veto/support.
    """

    clahe_img = apply_clahe(img)
    img = clahe_img.astype(np.float32) / 255.0

    v_sato = sato(
        img,
        sigmas=range(3, 12),
        black_ridges=False  # IMPORTANT
    )

    v_sato /= (v_sato.max() + 1e-8)

    # Transmission benefits from higher threshold
    thresh = threshold_otsu(v_sato)
    binary = v_sato > thresh * 1.2

    binary = remove_small_objects(binary, 200)
    binary = binary_closing(binary, disk(3))

    return binary.astype(bool), v_sato

def generate_mask_blue(img):
    """
    Blue channel: PRIMARY detector
    """

    clahe_img = apply_clahe(img)
    img = clahe_img.astype(np.float32) / 255.0

    v_sato = sato(
        img,
        sigmas=range(1, 8),
        black_ridges=True
    )

    v_sato /= (v_sato.max() + 1e-8)

    thresh = threshold_otsu(v_sato)
    binary = v_sato > thresh

    binary = remove_small_objects(binary, 100)
    binary = binary_closing(binary, disk(2))

    return binary.astype(bool), v_sato

def generate_mask_green(img):
    """
    Green channel pipeline.
    Used as a SUPPORT signal, not a primary detector.
    """

    clahe_img = apply_clahe(img)
    img = clahe_img.astype(np.float32) / 255.0

    # Slightly larger scales, green tends to show thicker features
    v_sato = sato(
        img,
        sigmas=range(2, 10),
        black_ridges=True
    )

    # Normalize for later fusion
    v_sato /= (v_sato.max() + 1e-8)

    thresh = threshold_otsu(v_sato)
    binary = v_sato > thresh * 1.1  # be stricter than blue

    binary = remove_small_objects(binary, 150)
    binary = binary_closing(binary, disk(2))

    return binary.astype(bool), v_sato

def combine_three(img_green, img_blue, img_trans):
    """
    Combines three channels in a physically meaningful way.
    Blue dominates geometry, others add confidence.
    """

    _, v_blue  = generate_mask_blue(img_blue)
    _, v_green = generate_mask_green(img_green)
    _, v_trans = generate_mask_trans(img_trans)

    # Weighted fusion (blue-dominant)
    v_fused = (
        0.6 * v_blue +
        0.25 * v_green +
        0.15 * v_trans
    )

    thresh = threshold_otsu(v_fused)
    binary = v_fused > thresh

    binary = remove_small_objects(binary, 120)
    binary = binary_closing(binary, disk(2))

    return binary.astype(bool), v_fused

def combine_three_strict(img_green, img_blue, img_trans):
    """
    Blue = detector
    Green + transmission = confidence veto
    """

    mask_blue, v_blue   = generate_mask_blue(img_blue)
    _, v_green          = generate_mask_green(img_green)
    _, v_trans          = generate_mask_trans(img_trans)

    confidence = (
        v_green +
        v_trans
    )

    confidence /= (confidence.max() + 1e-8)

    final = mask_blue & (confidence > 0.3)

    final = remove_small_objects(final, 120)
    final = binary_closing(final, disk(2))

    return final.astype(bool)


In [ ]:
img_blue = extract_square_glass_bmp("input_samples/0940 2024-07-29 14h17 pre (0,212-4915,7571) [BrightPolarizedBlue].bmp")
img_green = extract_square_glass_bmp("input_samples/0940 2024-07-29 14h17 pre (0,212-4915,7571) [BrightPolarizedGreen].bmp")
img_trans = extract_square_glass_bmp("input_samples/0940 2024-07-29 14h17 pre (0,212-4915,7571) [Transmission].bmp")
resized_img_blue = resize_image_to_side(img_blue, 1500)
resized_img_green = resize_image_to_side(img_green, 1500)
resized_img_trans = resize_image_to_side(img_trans, 1500)

#combined_binary, combined_sato = combine_three(resized_img_green, resized_img_blue, resized_img_trans)
combined_binary_strict = combine_three_strict(resized_img_green, resized_img_blue, resized_img_trans)
#cv2.imwrite("output_samples/0940 2024-07-29 14h17 pre (0,212-4915,7571) [Combined].bmp", to_uint8(combined_binary))
cv2.imwrite("output_samples/0940 2024-07-29 14h17 pre (0,212-4915,7571) [Combined_Strict].bmp", to_uint8(combined_binary_strict))


True

In [3]:
def isolate_cracks_from_origin(gray_uint8,
                               origin=None,
                               low_thr=10,          # very low: preserves faint lines
                               bridge_radius=1,     # small dilation to bridge gaps
                               seed_radius=4,       # seed disk at the origin
                               max_seed_radius=160, # grow if seed doesn’t touch lines
                               min_obj=20):
    """
    gray_uint8: grayscale image (0..255) where cracks are light on dark.
    origin: (y, x). If None, auto-estimate by densest neighborhood.
    Returns boolean mask of crack network connected to origin.
    """
    # 1) Gentle binarization (invert if needed so cracks are True and sparse)
    binary = gray_uint8 >= low_thr
    if binary.mean() > 0.5:
        binary = ~binary

    binary = remove_small_objects(binary, min_obj)

    # 2) Auto origin near densest junction if not given
    if origin is None:
        neigh = convolve(binary.astype(np.uint8), np.ones((3,3), np.uint8))
        origin = np.unravel_index(np.argmax(neigh), neigh.shape)

    # 3) Bridge tiny gaps
    bridged = binary_dilation(binary, disk(bridge_radius))

    # 4) Geodesic reconstruction (region growing from origin *inside* the crack mask)
    def grow(mask, origin, r0, rmax):
        for r in range(r0, rmax+1, 2):
            seed = np.zeros_like(mask, np.uint8)
            cv2.circle(seed, (int(origin[1]), int(origin[0])), r, 1, -1)
            seed = (seed > 0) & mask                  # must be <= mask for reconstruction
            if seed.any():
                recon = reconstruction(seed.astype(float), mask.astype(float),
                                       method='dilation') > 0
                if recon.any():
                    return recon, r
        return np.zeros_like(mask, bool), None

    recon, used_r = grow(bridged, origin, seed_radius, max_seed_radius)
    return recon, origin, used_r



In [ ]:
#Processing BrightPolarizedBlue Scan
img_blue = extract_square_glass_bmp("input_samples/0940 2024-07-29 14h17 pre (0,212-4915,7571) [BrightPolarizedBlue].bmp")
resized_img_blue = resize_image_to_side(img_blue, 1500)
img_blue_complete = to_uint8(generate_mask(resized_img_blue))
img_blue_incomplete = to_uint8(generate_mask_incomplete(resized_img_blue))
cv2.imwrite("output_samples/0940 2024-07-29 14h17 pre (0,212-4915,7571) [BrightPolarizedBlue][Complete].bmp", img_blue_complete)
cv2.imwrite("output_samples/0940 2024-07-29 14h17 pre (0,212-4915,7571) [BrightPolarizedBlue][Incomplete].bmp", img_blue_incomplete)

#Processing BrightPolarizedGreen Scan
img_green = extract_square_glass_bmp("input_samples/0940 2024-07-29 14h17 pre (0,212-4915,7571) [BrightPolarizedGreen].bmp")
resized_img_green = resize_image_to_side(img_green, 1500)
img_green_complete = to_uint8(generate_mask(resized_img_green))
img_green_incomplete = to_uint8(generate_mask(resized_img_green))
cv2.imwrite("output_samples/0940 2024-07-29 14h17 pre (0,212-4915,7571) [BrightPolarizedGreen][Complete].bmp", img_green_complete)
cv2.imwrite("output_samples/0940 2024-07-29 14h17 pre (0,212-4915,7571) [BrightPolarizedGreen][Incomplete].bmp", img_green_incomplete)

#Processing Transmission Scan
img_trans = extract_square_glass_bmp("input_samples/0940 2024-07-29 14h17 pre (0,212-4915,7571) [Transmission].bmp")
resized_img_trans = resize_image_to_side(img_trans, 1500)
img_trans_complete = to_uint8(generate_mask(resized_img_trans))
img_trans_incomplete = to_uint8(generate_mask(resized_img_trans))
cv2.imwrite("output_samples/0940 2024-07-29 14h17 pre (0,212-4915,7571) [Transmission][Complete].bmp", img_trans_complete)
cv2.imwrite("output_samples/0940 2024-07-29 14h17 pre (0,212-4915,7571) [Transmission][Incomplete].bmp", img_trans_incomplete)



True

In [5]:
#Testing area

# img = extract_square_glass_bmp("Proben/4.70.B.01/fracture/morphology/0940 2024-07-29 14h17 pre (0,212-4915,7571) [BrightPolarizedBlue].bmp",
#                                  margin=2, debug=False)

# cv2.imwrite("out/extracted_square.png", img)

# img = img.astype(np.float32) / 255.0

# v_sato = sato(img, sigmas=range(1, 8), black_ridges=True)
# v_frangi = frangi(img, sigmas=range(1, 8), black_ridges=True)

# cv2.imwrite("out/sato_output.png", normalize((v_sato * 255).astype(np.uint8)))
# cv2.imwrite("out/frangi_output.png", normalize((v_frangi * 255).astype(np.uint8)))

# plt.subplot(1,2,1)
# plt.imshow(v_sato, cmap='gray')
# plt.title('Sato')

# plt.subplot(1,2,2)
# plt.imshow(v_frangi, cmap='gray')
# plt.title('Frangi')

# plt.show()